In [1]:
import pandas as pd
import random

from faker import Faker

import sqlite3

%load_ext sql

In [2]:
N_PRODUTOS = 100
N_CLIENTES = 20
N_COMPRAS = 500

In [3]:
faker = Faker('pt_BR')
faker.name()

'Sra. Maria Eduarda Castro'

In [4]:
db = sqlite3.connect('TesteSQL.db')
db

In [5]:
%sql sqlite:///TesteSQL.db

In [6]:
produtos = pd.DataFrame([[f'prod {i:02}', round(random.random() * 10, 2)] 
                         for i in range(N_PRODUTOS)], 
                        columns=['produto', 'preço'])
produtos.head()

,produto,preço
0,prod 00,2.89
1,prod 01,6.60
2,prod 02,8.23
3,prod 03,5.87
4,prod 04,1.10


In [7]:
produtos.to_sql('produtos', db, index_label='id', if_exists='replace')

In [8]:
pd.read_sql("SELECT * FROM produtos;", db, 'id').head()

,produto,preço
id,,
0,prod 00,2.89
1,prod 01,6.60
2,prod 02,8.23
3,prod 03,5.87
4,prod 04,1.10


In [9]:
clientes = pd.DataFrame([
    [faker.name(), faker.free_email(), random.randint(18, 60)]
for _ in range(N_CLIENTES)], columns=['nome', 'email', 'idade'])

clientes.head()

,nome,email,idade
0,Clara Melo,nalmeida@ig.com.br,32
1,João Vitor da Rocha,moraesana-luiza@hotmail.com,47
2,Maria Eduarda Teixeira,cardosoerick@yahoo.com.br,32
3,Lucca Teixeira,vcampos@ig.com.br,32
4,Juan Moura,santoseduardo@hotmail.com,50


In [10]:
clientes.to_sql('clientes', db, index_label='id', if_exists='replace')
pd.read_sql("SELECT * FROM clientes;", db, 'id').head()

,nome,email,idade
id,,,
0,Clara Melo,nalmeida@ig.com.br,32
1,João Vitor da Rocha,moraesana-luiza@hotmail.com,47
2,Maria Eduarda Teixeira,cardosoerick@yahoo.com.br,32
3,Lucca Teixeira,vcampos@ig.com.br,32
4,Juan Moura,santoseduardo@hotmail.com,50


In [11]:
compras = pd.DataFrame([[
    random.randint(0, N_CLIENTES - 1),
    random.randint(0, N_PRODUTOS - 1)
] for _ in range(N_COMPRAS)], columns=['id_cliente', 'id_produto'])
compras.head()

,id_cliente,id_produto
0,14,99
1,6,19
2,7,3
3,9,67
4,2,38


In [12]:
compras.to_sql('compras', db, index_label='id', if_exists='replace')
pd.read_sql("SELECT * FROM compras;", db, 'id').head()

,id_cliente,id_produto
id,,
0,14,99
1,6,19
2,7,3
3,9,67
4,2,38


In [13]:
del compras, clientes, produtos

# Select

Ao fazer uma extração num banco de dados SQL, independentemente do 
DBMS (se é Oracle, SQLite, MySQL/MariaDB e etc.), você sempre vai utilizar a 
palava `SELECT` seguida pelo que você quer trazer, e por fim a palavra FROM
informando de onde você irá coletar esses dados, por exemplo:

In [14]:
%%sql

SELECT nome, idade FROM clientes

 * sqlite:///TesteSQL.db
Done.


nome,idade
Clara Melo,32
João Vitor da Rocha,47
Maria Eduarda Teixeira,32
Lucca Teixeira,32
Juan Moura,50
Ana Luiza Moura,47
Sr. Leonardo Correia,31
João Guilherme da Cunha,21
Olivia Duarte,22
Elisa das Neves,51


Nesse caso, eu estou trazendo o nome e idade de todos os clientes da base.

# Limit

Geralmente não queremos trazer todos os dados de uma vez, muitas vezes os dados
que guardamos nas tabelas do banco de dados iriam facilmente estourar a  memória
dos nossos servidores, para limitar a pesquisa, existem vários métodos, o mais 
simples que tem é o `LIMIT n`, onde você informa quantos registros você quer ver
no máximo.

In [15]:
%%sql

SELECT * FROM clientes LIMIT 5

 * sqlite:///TesteSQL.db
Done.


id,nome,email,idade
0,Clara Melo,nalmeida@ig.com.br,32
1,João Vitor da Rocha,moraesana-luiza@hotmail.com,47
2,Maria Eduarda Teixeira,cardosoerick@yahoo.com.br,32
3,Lucca Teixeira,vcampos@ig.com.br,32
4,Juan Moura,santoseduardo@hotmail.com,50


Nesse caso, eu trouxe 5 registros da tabela cliente.

<span style="color:red"><b>Em uma aplicação, sempre use o</span></b> `LIMIT` 
<span style="color:red"><b>com um</span></b> `ORDER BY` 
<span style="color:red"><b>quer será explicado mais a frente

## Exercício 01

Traga 10 produtos e seu respectivo preço

In [16]:
%%sql

SELECT * FROM produtos LIMIT 10

 * sqlite:///TesteSQL.db
Done.


id,produto,preço
0,prod 00,2.89
1,prod 01,6.6
2,prod 02,8.23
3,prod 03,5.87
4,prod 04,1.1
5,prod 05,6.76
6,prod 06,4.18
7,prod 07,6.29
8,prod 08,2.73
9,prod 09,6.18


# Projeção

Também posso operar sobre os valores retornados, por exemplo, se ao invés da idade
da pessoa, eu quiser saber somente a faixa etária dela, posso fazer algum cálculo
no select e assim trazer esse dado já trabalhado do próprio banco de dados. 

In [17]:
%%sql

SELECT NOME, ROUND(idade / 10) * 10 AS FAIXA_ETARIA
FROM clientes

 * sqlite:///TesteSQL.db
Done.


nome,FAIXA_ETARIA
Clara Melo,30.0
João Vitor da Rocha,40.0
Maria Eduarda Teixeira,30.0
Lucca Teixeira,30.0
Juan Moura,50.0
Ana Luiza Moura,40.0
Sr. Leonardo Correia,30.0
João Guilherme da Cunha,20.0
Olivia Duarte,20.0
Elisa das Neves,50.0


Essa ação de selecionar um dado trabalhado, recebe vários nomes, como mapeamento
ou projeção.

## Exercício 2

Traga 10 produtos e seu preço convertido para dólar, considere a cotação do dólar
com sendo US\\$1,00=R\\$5,20

In [18]:
%%sql

SELECT produto, (preço/5.20)  AS preço_convertido
FROM produtos LIMIT 10

 * sqlite:///TesteSQL.db
Done.


produto,preço_convertido
prod 00,0.5557692307692308
prod 01,1.2692307692307692
prod 02,1.5826923076923076
prod 03,1.1288461538461538
prod 04,0.21153846153846154
prod 05,1.2999999999999998
prod 06,0.8038461538461538
prod 07,1.2096153846153845
prod 08,0.525
prod 09,1.1884615384615385


# Where

O `LIMIT` como dito anteriormente é a forma mais simples de reduzir a quantidade
de dados trazidos em uma pesquisa, mas a forma mais comum de se usar é o `WHERE`.

Geralmente quando fazemos uma consulta no banco de dados, não precisamos de todos
os dados da tabela, somente dados que respeitem algum requisito, como por exemplo,
no nosso caso, se a gente quiser saber quais clientes temos com menos de 30 anos:

In [19]:
%%sql

SELECT nome, idade FROM clientes WHERE idade < 30

 * sqlite:///TesteSQL.db
Done.


nome,idade
João Guilherme da Cunha,21
Olivia Duarte,22
João Felipe Costa,23
Júlia Duarte,21
Henrique da Luz,27


## Exercício 3

Traga todos os produtos que custem menos do que 1 real

In [20]:
%%sql

SELECT produto, preço FROM produtos WHERE preço<1

 * sqlite:///TesteSQL.db
Done.


produto,preço
prod 24,0.4
prod 29,0.41
prod 30,0.71
prod 39,0.35
prod 44,0.03
prod 47,0.78
prod 59,0.63
prod 85,0.81
prod 92,0.04
prod 94,0.87


# Join

Uma das principais características dos bancos relacionais é justamente a sua capacidade
de ter relacionamento entre os dados.

Geralmente esse relacionamento é feito utilizando chaves primárias (PK). Geralmente
as PKs são representadas por um número inteiro chamado ID, mas não necessariamente,
por exemplo, um cadastro de pessoas pode ter como PK o CPF da pessoa, o que importa
é que não haja repetição de valores.

As PKs também podem ter mais de uma coluna, por exemplo, um supermercado pode ter
como PK um produto e a marca dele.

Mas por hora iremos abstrair essas PKs menos ortodoxas e focar nas padrões que é só
uma coluna ID.

As PKs geralmente são utilizadas para formar relacionamento entre as tabelas, ou seja,
a partir do momento que eu cadastrei um cliente no meu sistema, e gerei um ID para ele
toda as interações dele serão registradas utilizando o seu ID e não mais dados
pessoais.

No nosso caso, registramos as compras utilizando o ID do cliente e do produto, como
podemos ver abaixo

In [21]:
%%sql

SELECT cl.*, po.*
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
LIMIT 5

 * sqlite:///TesteSQL.db
Done.


id,nome,email,idade,id_1,produto,preço
14,Pedro Miguel Vieira,mbarbosa@uol.com.br,57,99,prod 99,3.52
6,Sr. Leonardo Correia,vitor-gabrielpinto@hotmail.com,31,19,prod 19,5.76
7,João Guilherme da Cunha,ryancardoso@uol.com.br,21,3,prod 03,5.87
9,Elisa das Neves,lara55@yahoo.com.br,51,67,prod 67,3.89
2,Maria Eduarda Teixeira,cardosoerick@yahoo.com.br,32,38,prod 38,5.22


A tabela de compras traz consigo o ID do cliente que fez a compra e o ID do produto
comprado, ocupando assim muito menos espaço do que se eu tivesse uma tabela com
todos os dados.

> NOTA: Quando nas colunas selecionadas faço algo do tipo alias.* eu estou trazendo todas as colunas daquela tabel

> NOTA: Em 99% dos casos, independetemente do intuito do SELECT, o relacionamento entre as tabelas vai utilizar as mesmas colunas que usou em SELECTs anteriores

## Exercício 4

Traga 10 compras do produto `prod 02`

In [22]:
%%sql

SELECT cl.*, po.*
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
WHERE produto = 'prod 02'
LIMIT 10

 * sqlite:///TesteSQL.db
Done.


id,nome,email,idade,id_1,produto,preço
12,Sabrina Costela,nmoraes@uol.com.br,55,2,prod 02,8.23
11,Sr. Pietro Fogaça,hda-rosa@bol.com.br,41,2,prod 02,8.23
9,Elisa das Neves,lara55@yahoo.com.br,51,2,prod 02,8.23
6,Sr. Leonardo Correia,vitor-gabrielpinto@hotmail.com,31,2,prod 02,8.23
3,Lucca Teixeira,vcampos@ig.com.br,32,2,prod 02,8.23
18,Enzo Moura,frodrigues@bol.com.br,30,2,prod 02,8.23
13,Luiz Otávio Monteiro,yasmincostela@bol.com.br,48,2,prod 02,8.23


# Distinct

Muitas vezes as nossas consultas nos trazem resultados repetidos, quando a repetição
não nos importa, por exemplo, mesmo na nossa base tendo 20 clientes, não necessariamente
todos efetuaram compras, se eu quiser saber todos que o fizeram posso fazer:

In [23]:
%%sql

SELECT DISTINCT cl.nome
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID

 * sqlite:///TesteSQL.db
Done.


nome
Pedro Miguel Vieira
Sr. Leonardo Correia
João Guilherme da Cunha
Elisa das Neves
Maria Eduarda Teixeira
Sr. Pietro Fogaça
Clara Melo
Sr. Enzo Martins
Henrique da Luz
Juan Moura


Como podemos ver, apesar de eu ter feito {{N_COMPRAS}} compras, o meu resultado
não traz essa quantidade de linhas, traz exatamente a quantidade de clientes 
que fizeram compras.

## Exercício 5

Liste todos clientes que compraram o produto `prod 02`

In [24]:
%%sql

SELECT DISTINCT cl.nome
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
WHERE produto = 'prod 02'


 * sqlite:///TesteSQL.db
Done.


nome
Lucca Teixeira
Sr. Leonardo Correia
Elisa das Neves
Sr. Pietro Fogaça
Sabrina Costela
Luiz Otávio Monteiro
Enzo Moura


# Redução

No SQL também são comuns funções de redução, elas percorrem todas as linhas de uma
determinada coluna aplicando uma função de redução (somatório, contagem e etc.)
e retornam somente uma linha ao invés de todas as linhas lidas por ela, por exemplo
se eu quiser saber quantas compras do `prod 02` foram feitas, posso fazer:

In [25]:
%%sql

SELECT count(*)
FROM compras AS co
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
WHERE po.produto = 'prod 02'

 * sqlite:///TesteSQL.db
Done.


count(*)
7


Existem outras funções como SUM, AVG e etc., você pode sempre consultar a 
[documentação oficial](https://www.sqlite.org/lang_aggfunc.html) para ver 
todas elas

## Exercício 6

Qual foi a receita total?

In [26]:
%%sql

SELECT round(sum(po.preço)) AS receita_total
FROM compras AS co
JOIN produtos as po
ON co.ID_PRODUTO = po.ID

 * sqlite:///TesteSQL.db
Done.


receita_total
2290.0


# Group by

As funções de agregação ficam ainda mais fortes quando agrupamos valores.
Isto é, elas mostram todo o seu potencial quando criamos pequenos grupos de dados,
ao invés de contarmos por exemplo todas as vendas do dia, podemos aplicar contagem
produto a produto com somente uma query, como mostrado abaixo:

In [27]:
%%sql

SELECT po.produto, count(*)
FROM compras AS co
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by po.produto
limit 10

 * sqlite:///TesteSQL.db
Done.


produto,count(*)
prod 00,5
prod 01,12
prod 02,7
prod 03,6
prod 04,4
prod 05,4
prod 06,9
prod 07,4
prod 08,2
prod 09,2


Dessa forma, consigo fazer uma contagem diferente para cada produto, sem precisar
fazer diversas querys, algo muito útil.

## Exercício 7

Quanto cada cliente gastou?

In [28]:
%%sql

SELECT cl.nome, round(sum(po.preço)) as gasto
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by cl.nome


 * sqlite:///TesteSQL.db
Done.


nome,gasto
Ana Luiza Moura,46.0
Clara Melo,102.0
Elisa das Neves,106.0
Enzo Moura,87.0
Henrique da Luz,101.0
João Felipe Costa,135.0
João Guilherme da Cunha,163.0
João Vitor da Rocha,127.0
Juan Moura,92.0
Júlia Duarte,77.0


# Order by

Muitas vezes queremos os dados em ordem específicas, por exemplo, ao visitar uma
loja online, você não espera ver produtos caros e desinteressantes sendo exibidos
para você aleatoriamente, você espera ver em uma ordem lógica, sejam os mais
baratos primeiros, os mais relevantes para o seu perfil, ou até alfabéticamente.


Tudo isso pode ser feito utilizando o comando `ORDER BY`, mas misturado com
outras operações como o `LIMIT` ele pode fazer coisas ainda melhores, como
por exemplo, trazer os 10 produtos mais caros:

In [29]:
%%sql

SELECT produto, preço
FROM produtos
ORDER BY preço DESC
LIMIT 10

 * sqlite:///TesteSQL.db
Done.


produto,preço
prod 54,9.83
prod 93,9.59
prod 21,9.58
prod 81,9.48
prod 63,9.46
prod 95,9.45
prod 41,9.42
prod 32,9.34
prod 22,9.24
prod 65,9.11


Com isso podemos ver o poder do `ORDER BY` + `LIMIT` para trazer os N primeiros
registros de uma determinada métrica

## Exercício 8

Traga os 10 clientes que mais gastaram

In [30]:
%%sql

SELECT cl.nome
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by cl.nome
order by sum(po.preço)
LIMIT 10

 * sqlite:///TesteSQL.db
Done.


nome
Ana Luiza Moura
Júlia Duarte
Enzo Moura
Juan Moura
Sr. Enzo Martins
Henrique da Luz
Clara Melo
Elisa das Neves
Sabrina Costela
Pedro Miguel Vieira


O order by junto do limit é extremamente importante para fazer paginação de dados,
por exemplo, novamente ao acessar o site de compras, os produtos não são todos
exibidos de uma vez para você, do contrário, a memória do seu computador iria
para o espaço, para evitar isso, tem aqueles botões na parte de baixo para ver
a próxima página, e com isso, você pode avançar e recuar.

Digamos que um cliente viu os 10 produtos mais caros e não se interessou por
nenhum e quer ver a próxima página, ele pode fazer:

In [31]:
%%sql

SELECT produto, preço
FROM produtos
ORDER BY preço DESC
LIMIT 10,10

 * sqlite:///TesteSQL.db
Done.


produto,preço
prod 18,8.75
prod 68,8.67
prod 89,8.52
prod 15,8.31
prod 28,8.25
prod 02,8.23
prod 84,8.0
prod 50,7.9
prod 74,7.88
prod 73,7.86


O primeiro 10, representa de onde começa e o segundo quantos registros vai trazer.
Você pode mexer nos valores acima e testar a vontade para ver como muda.

## Exercício 9

Traga o 11º cliente que mais gastou

In [32]:
%%sql

SELECT cl.nome
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by cl.nome
order by sum(po.preço)
LIMIT 10,01

 * sqlite:///TesteSQL.db
Done.


nome
Sr. Pietro Fogaça


# Funções janeladas

As funções janeladas ou funções analíticas como também são conhecidas, são funções
que se aplicam a todas a coluna trazendo métricas como qual posição essa pessoa está
entre as que mais gastaram, ou quantas pessoas no mínimo foram necessárias para 
alcançar até 25% da receita total.

In [33]:
%%sql

SELECT produto, preço, rank() over (order by preço desc) as posição
FROM produtos
order by posição
limit 10

 * sqlite:///TesteSQL.db
Done.


produto,preço,posição
prod 54,9.83,1
prod 93,9.59,2
prod 21,9.58,3
prod 81,9.48,4
prod 63,9.46,5
prod 95,9.45,6
prod 41,9.42,7
prod 32,9.34,8
prod 22,9.24,9
prod 65,9.11,10


Você pode usar as funções janeladas quando lhe for conveniente, mas não serão
obrigatórias em nenhum exercício

## Exercício 10

Qual foi o produto que menos vendeu?

In [34]:
%%sql
SELECT produto, rank() over (order by sum(po.preço) desc ) as menos_vendido
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID

 * sqlite:///TesteSQL.db
Done.


produto,menos_vendido
prod 99,1


# Operadores lógicos comuns

Alguns operadores lógicos bem famosos são o `=`, `>`, `<`, `>=`, `<=`, `!=`,
mas existem outros menos comuns, como o `LIKE` e `IN`.

O primeiro, serve para fazer pesquisas simples dentro de texto, ele irá verificar
caracter a caracter se a string informada bate com o padrão, de forma mais prática

```SQL
'abc' LIKE 'abc' -- Verdadeiro

'abcd' LIKE 'abc' -- Falso

'cbc' LIKE 'abc' -- Falso

'dabc' LIKE 'abc' -- Falso
```

Até aí é exatamente igual ao operador `=`, mas quando adicionamos os wildcards
`%` e `_` que o `LIKE` mostra o seu valor.

O wildcard `_` pode ser usado para representar qualquer caracter, por exemplo:

```SQL
'abc' LIKE 'ab_' -- Verdadeiro
'abd' LIKE 'ab_' -- Verdadeiro
'ab0' LIKE 'ab_' -- Verdadeiro
'ab ' LIKE 'ab_' -- Verdadeiro
'abcd' LIKE 'ab_' -- Falso
'cbc' LIKE 'ab_' -- Falso
'ab' LIKE 'ab_' -- Falso
'dabc' LIKE 'ab_' -- Falso
```

Já o wildcard `%` pode ser usado para representar qualquer sequência de caracteres,
por exemplo:

```SQL
'abc' LIKE 'ab%' -- Verdadeiro
'abd' LIKE 'ab%' -- Verdadeiro
'ab0' LIKE 'ab%' -- Verdadeiro
'ab ' LIKE 'ab%' -- Verdadeiro
'abcd' LIKE 'ab%' -- Verdadeiro
'cbc' LIKE 'ab%' -- Falso
'ab' LIKE 'ab%' -- Verdadeiro
'dabc' LIKE 'ab%' -- Falso
```

Usando o nosso exemplo, vamos pegar todos os clientes que tenham e-mail uol:

In [35]:
%%sql

SELECT *
FROM CLIENTES
WHERE EMAIL LIKE '%@uol.com.br'

 * sqlite:///TesteSQL.db
Done.


id,nome,email,idade
7,João Guilherme da Cunha,ryancardoso@uol.com.br,21
12,Sabrina Costela,nmoraes@uol.com.br,55
14,Pedro Miguel Vieira,mbarbosa@uol.com.br,57


Já o operador `IN` serve para verificar se o valor está dentro de um cojunto, por exemplo

```SQL
1 in (1, 2, 3) -- Verdadeiro
2 in (1, 2, 3) -- Verdadeiro
3 in (1, 2, 3) -- Verdadeiro
4 in (1, 2, 3) -- Falso
(1, 2) in ((1, 2), (3, 4)) -- Verdadeiro
(1, 3) in ((1, 2), (3, 4)) -- Falso
```

In [36]:
%%sql

SELECT *
FROM PRODUTOS
WHERE PRODUTO IN ('prod 01', 'prod 20', 'batata')

 * sqlite:///TesteSQL.db
Done.


id,produto,preço
1,prod 01,6.6
20,prod 20,4.82


# Subquerys

Subquerys são utilizadas para fazer extrações mais complexas, quando para 
alcançar um resultado, é necessário outro resultado, digamos, listar todos os
produtos comprados pelo cliente que mais gastou, primeiro precisamos encotrar o 
cliente que mais gastou e depois os produtos que ele comprou.

Podemos separar isso em duas querys:

1. Selecionar o cliente que mais gastou
2. Selecionar todos os produtos que um cliente comprou

E elas ficariam como:

In [37]:
%%sql

-- Selecionando o cliente que mais gastou

SELECT cl2.ID, cl2.nome, sum(po2.preço) AS GASTO
FROM clientes cl2
JOIN compras AS co2
ON co2.ID_CLIENTE = cl2.ID
JOIN produtos as po2
ON co2.ID_PRODUTO = po2.ID
group by cl2.nome, cl2.id
ORDER BY gasto DESC
LIMIT 1

 * sqlite:///TesteSQL.db
Done.


id,nome,GASTO
7,João Guilherme da Cunha,162.75999999999996


In [38]:
%%sql

-- Selecionar todos os produtos que umm cliente comprou

SELECT DISTINCT po.produto
FROM produtos as po
JOIN compras AS co
ON co.ID_PRODUTO = po.ID
WHERE co.ID_CLIENTE IN (5)

 * sqlite:///TesteSQL.db
Done.


produto
prod 10
prod 23
prod 47
prod 29
prod 60
prod 80
prod 35
prod 85
prod 06
prod 32


In [39]:
%%sql

-- Juntando as duas:
    
SELECT DISTINCT po.produto
FROM produtos as po
JOIN compras AS co
ON co.ID_PRODUTO = po.ID
WHERE co.ID_CLIENTE IN (
    SELECT cl2.ID
    FROM clientes cl2
    JOIN compras AS co2
    ON co2.ID_CLIENTE = cl2.ID
    JOIN produtos as po2
    ON co2.ID_PRODUTO = po2.ID
    group by cl2.nome, cl2.id
    ORDER BY sum(po2.preço) DESC
    LIMIT 1
)

 * sqlite:///TesteSQL.db
Done.


produto
prod 03
prod 91
prod 95
prod 74
prod 25
prod 40
prod 17
prod 73
prod 80
prod 82


Se eu quisesse exibir o nome do cliente, eu poderia fazer mais um join pegando
o nome dele na tabela cliente, mas nesse caso, eu estou pegando um dado que eu
consigo facilmente na subquery, existem formas de trazer dados lá de dentro,
como por  exemplo:

In [40]:
%%sql

    
SELECT tb.nome, po.produto
FROM produtos as po
JOIN compras AS co
ON co.ID_PRODUTO = po.ID
join (
    SELECT cl2.ID, cl2.nome
    FROM clientes cl2
    JOIN compras AS co2
    ON co2.ID_CLIENTE = cl2.ID
    JOIN produtos as po2
    ON co2.ID_PRODUTO = po2.ID
    group by cl2.nome, cl2.id
    ORDER BY sum(po2.preço) DESC
    LIMIT 1
) tb on co.ID_CLIENTE = tb.ID
group by tb.nome, po.produto

 * sqlite:///TesteSQL.db
Done.


nome,produto
João Guilherme da Cunha,prod 03
João Guilherme da Cunha,prod 05
João Guilherme da Cunha,prod 12
João Guilherme da Cunha,prod 17
João Guilherme da Cunha,prod 18
João Guilherme da Cunha,prod 25
João Guilherme da Cunha,prod 28
João Guilherme da Cunha,prod 37
João Guilherme da Cunha,prod 40
João Guilherme da Cunha,prod 41


Não existe limite de quantidade de subquerys que você pode fazer, sejam uma duas
ou várias, fazendo uma verdadeira boneca russa, mas use com atenção, verifique
sempre se realmente é necessário.

Se por exemplo, eu quisesse saber quantos clientes compraram os mesmos
produtos que a pessoa que mais gastou comprou


1. Selecionar o cliente que mais gastou
2. Selecionar todos os produtos que um cliente comprou
3. Contar quantos clientes compraram um produto

Como a query 1 e 2 já são antigas, vou fazer a 3

In [41]:
%%sql

-- Contar quantos clientes compraram um produto

SELECT po.produto, count(DISTINCT co.ID_CLIENTE)
FROM compras co
JOIN produtos po on po.ID = co.ID_PRODUTO
WHERE co.ID_PRODUTO IN (5)
GROUP by po.produto

 * sqlite:///TesteSQL.db
Done.


produto,count(DISTINCT co.ID_CLIENTE)
prod 05,4


In [42]:
%%sql

SELECT po.produto, count(DISTINCT co.ID_CLIENTE)
FROM compras co
JOIN produtos po on po.ID = co.ID_PRODUTO
WHERE co.ID_PRODUTO IN (
    SELECT DISTINCT po2.id
    FROM produtos as po2
    JOIN compras AS co2
    ON co2.ID_PRODUTO = po2.ID
    WHERE co2.ID_CLIENTE IN (
        SELECT cl3.ID
        FROM clientes cl3
        JOIN compras AS co3
        ON co3.ID_CLIENTE = cl3.ID
        JOIN produtos as po3
        ON co3.ID_PRODUTO = po3.ID
        group by cl3.nome, cl3.id
        ORDER BY sum(po3.preço) DESC
        LIMIT 1
    )
)
GROUP by po.produto

 * sqlite:///TesteSQL.db
Done.


produto,count(DISTINCT co.ID_CLIENTE)
prod 03,5
prod 05,4
prod 12,6
prod 17,4
prod 18,3
prod 25,6
prod 28,6
prod 37,5
prod 40,6
prod 41,5


Mas novamente podemos simplificar usando join ao invés de where, lembre-se,
quanto menos vezes você repetir o nome de uma tabela, melhor

In [43]:
%%sql

SELECT po.produto, count(DISTINCT co.ID_CLIENTE)
FROM compras co
JOIN produtos as po on co.ID_PRODUTO = po.ID
JOIN (
    SELECT cl3.ID
    FROM clientes cl3
    JOIN compras AS co3
    ON co3.ID_CLIENTE = cl3.ID
    JOIN produtos as po3
    ON co3.ID_PRODUTO = po3.ID
    group by cl3.nome, cl3.id
    ORDER BY sum(po3.preço) DESC
    LIMIT 1
) q1
JOIN(
    SELECT co2.id_PRODUTO, co2.ID_CLIENTE
    FROM compras AS co2
    GROUP BY co2.id_PRODUTO, co2.ID_CLIENTE
) q2 ON co.ID_PRODUTO = q2.ID_PRODUTO AND q2.ID_CLIENTE = q1.ID
GROUP by po.produto

 * sqlite:///TesteSQL.db
Done.


produto,count(DISTINCT co.ID_CLIENTE)
prod 03,5
prod 05,4
prod 12,6
prod 17,4
prod 18,3
prod 25,6
prod 28,6
prod 37,5
prod 40,6
prod 41,5


# With

Muitas vezes fica confuso ficar colocando várias subquerys no meio da query 
pricipal, ou você quer repetir ela, não quer ter que trocar alias e etc.

Para resolver esses problemas, ou até outros, consulte a 
[documentação](https://www.sqlite.org/lang_with.html) para ver
tudo o que ele é capaz, existe o `WITH`.

In [44]:
%%sql

WITH q1 AS (
    SELECT cl.ID
    FROM clientes cl
    JOIN compras AS co
    ON co.ID_CLIENTE = cl.ID
    JOIN produtos as po
    ON co.ID_PRODUTO = po.ID
    group by cl.nome, cl.id
    ORDER BY sum(po.preço) DESC
    LIMIT 1
),
q2 AS (
    SELECT co.id_PRODUTO, co.ID_CLIENTE
    FROM compras AS co
    GROUP BY co.id_PRODUTO, co.ID_CLIENTE
)
SELECT po.produto, count(DISTINCT co.ID_CLIENTE)
FROM compras co
JOIN produtos as po on co.ID_PRODUTO = po.ID
JOIN q1
JOIN q2 ON co.ID_PRODUTO = q2.ID_PRODUTO AND q2.ID_CLIENTE = q1.ID
GROUP by po.produto

 * sqlite:///TesteSQL.db
Done.


produto,count(DISTINCT co.ID_CLIENTE)
prod 03,5
prod 05,4
prod 12,6
prod 17,4
prod 18,3
prod 25,6
prod 28,6
prod 37,5
prod 40,6
prod 41,5


## Exercício 11

Traga todos os clientes que compraram o produto que menos vendeu

In [45]:
%%sql


    
SELECT DISTINCT cl.nome, po.produto      #todos os clientes que compraram
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID 
WHERE co.ID_PRODUTO IN (             #colocar o mesmo no select
SELECT  co2.ID_PRODUTO              #produto que menos vendeu
    FROM clientes cl2
    JOIN compras AS co2
    ON co2.ID_CLIENTE = cl2.ID
    JOIN produtos as po2
    ON co2.ID_PRODUTO = po2.ID
    group by po2.produto
    order by  count(*)
    limit 1
)

 * sqlite:///TesteSQL.db
Done.


nome,produto
Luiz Otávio Monteiro,prod 33


## Exercício 12

Traga quanto gastou cada cliente que comprou o produto que menos vendeu

In [46]:
%%sql

WITH quantidade_de_produto_mv as (         
	SELECT count(*) as qtd
	FROM compras AS co
	group by co.id_produto
	order by qtd
	limit 1	
),
produto_que_menos_vendeu AS (                                 #produto que menos vendeu
	SELECT co.ID_PRODUTO, count(*) as qtd
	FROM compras AS co
	group by co.id_produto
	having qtd = (SELECT qtd from quantidade_de_produto_mv)
),
oq_cd_1_comprou as (                               #o que cada cliente comprou
	SELECT
		cl.nome, 
		co.id_cliente,
		po.produto,
		co.id_produto,
		po.preço 
	FROM clientes cl
	JOIN compras AS co ON co.ID_CLIENTE = cl.ID
	JOIN produtos as po ON co.ID_PRODUTO = po.ID
), quanto_cada_um_gastou as (                          #quanto gastou cada cliente
   SELECT 
    id_cliente,
    nome,
 	sum(preço) as gasto
   FROM oq_cd_1_comprou
   group by id_cliente, nome
 )
 , quem_comprou_menos_vendeu as (               #quem comprou o que menos vendeu
	select 
 	DISTINCT nome, id_cliente 
 	FROM oq_cd_1_comprou as occ
 	JOIN produto_que_menos_vendeu as pmv ON occ.id_produto = pmv.id_produto
 )
  SELECT                                  #quanto gastou cada cliente que comprou o produto que menos vendeu
  qcmv.nome, qcmv.id_cliente, gasto
  FROM quanto_cada_um_gastou as qcug
  JOIN quem_comprou_menos_vendeu as qcmv ON   qcug.id_cliente = qcmv.id_cliente

 * sqlite:///TesteSQL.db
Done.


nome,id_cliente,gasto
Sabrina Costela,12,109.01000000000002
Luiz Otávio Monteiro,13,143.44000000000003


## Exercício 13

Quanto em média os clientes gastaram?

In [74]:
%%sql

SELECT avg(po.preço) as média
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID


 * sqlite:///TesteSQL.db
Done.


média
4.579399999999996


## Exercício 14

Quanto em média gastou cada cliente?

In [59]:
%%sql

SELECT cl.nome, avg(po.preço) as média
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by cl.nome
order by avg(po.preço)

 * sqlite:///TesteSQL.db
Done.


nome,média
Ana Luiza Moura,2.7282352941176473
Sr. Pietro Fogaça,3.39060606060606
Enzo Moura,3.4759999999999995
Sr. Enzo Martins,3.777692307692308
Juan Moura,3.989130434782609
Lucca Teixeira,3.9896874999999996
Júlia Duarte,4.03
Henrique da Luz,4.214166666666667
Pedro Miguel Vieira,4.584583333333333
Vitor Gabriel Pereira,4.815769230769231


## Exercício 15

Quantos clientes tem cada faixa etária (Agrupe por faixa de 10 anos)

i.e.: \[18 -> 19\], \[20 -> 29\], \[30 -> 39\], ...

In [70]:
%%sql

SELECT nome, 
CASE
    WHEN idade <=10 THEN '0 - 10'
    WHEN idade <=20 THEN '11 - 20'
    WHEN idade <=30 THEN '21 - 30'
    WHEN idade <=40 THEN '31 - 40'
    ELSE 
END AS faixa_etaria
FROM clientes
GROUP BY faixa_etaria

 * sqlite:///TesteSQL.db
(sqlite3.OperationalError) no such column: x
[SQL: SELECT nome, CASE WHEN idade <=10 THEN '0 - 10'
    WHEN idade <=20 THEN '11 - 20'
    WHEN idade <=30 THEN '21 - 30'
    WHEN idade <=40 THEN '31 - 40'
    ELSE x
END AS faixa_etaria
FROM clientes
GROUP BY faixa_etaria]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


## Exercício 16

Qual é o gasto médio para cada faixa etária?

In [50]:
%%sql



 * sqlite:///TesteSQL.db


## Exercício 17

Quais foram os 5 produtos que mais venderam na faixa etária que teve maior
gasto médio?

In [51]:
%%sql



 * sqlite:///TesteSQL.db


## Exercício 18

Quais foram os 5 produtos que mais venderam na faixa etária que teve maior
gasto absoluto?

In [52]:
%%sql



 * sqlite:///TesteSQL.db


# Desafio

Para cada compra acima de 5 reais, os clientes vão ganhar um número para o
sorteio para cada 5 reais, salve esse número no banco e associe com o cliente

Exemplo, eu gastei 20 reais, vou ganhar 4 números, por exemplo 20, 21, 22 e 23

O primeiro lugar no sorteio vai ser um carro, o segundo uma geladeira
e o terceiro um microondas

E então sorteie e imprima na tela um e-mail para os três vencedores no formato:


```
Para : [e-mail]
Assunto: Parabéns, o número [número] foi sortedo

Corpo: Parabéns Sr(a). [nome], você acaba de ganhar um(a) [item] no nosso sorteio
```
Para esse desafio você pode usar a linguagem `Python` além do `SQL`, 
mas tente usar o banco sempre que possível

Lembre-se do

```python
df = pd.read_sql('SELECT * FROM clientes', db)
```